In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd src/py

/src/py


In [3]:
import sys
import json
import argparse
import time

import build_time_dist_matrix
import optimization
import visualization
import schedule
import upload_results
import manual_viz

In [4]:
from config.config_manager import ConfigManager, ConfigFileLocations, GPSInputPaths, ManualEditsInputPaths

In [5]:
cp ../../build_parameters.yml /src/py/

In [6]:
ls

__init__.py*                geojson_to_gpx_converter.py*  readme.md*
__pycache__/                main_application.py*          run_routing.py*
build_parameters.yml*       manage_aws.py*                schedule.py*
build_time_dist_matrix.py*  manual_viz.py*                scratchpad.py*
cloud_context.py*           node_data_pickle.py*          server.py*
config/                     optimization.py*              ui/
elevation_utils.py*         osrm_text_instructions.py*    upload_results.py*
file_config.py*             output/                       visualization.py*


In [7]:
config_manager =  ConfigManager.load_from_local(
            '/data/', False, None)

INFO:root:Missing ('Closed', 'closed'), adding substition


In [8]:
import logging

In [9]:
routing_config = config_manager.get_routing_config()
logging.info('Building Time/Distance Matrices')
# check if node_loader_options are specified
config = routing_config.get_raw_json()
if 'node_loader_options' in config.keys():
    node_data = build_time_dist_matrix.process_nodes(
        config_manager,
        config['node_loader_options'],
        config['zone_configs'])

INFO:root:Building Time/Distance Matrices
INFO:root:Num buckets Assumed: 2


In [10]:
solution = optimization.run_optimization(node_data, config)
visualization_data = visualization.create_visualizations(solution)    

False


INFO:root:Took 1.80567 seconds to resequence
INFO:root:Total Distance of all routes: 15.922000000000002km
INFO:root:Total Time of all routes: 1736.9000000000003min


False


INFO:root:Took 2.871233 seconds to resequence
INFO:root:Total Distance of all routes: 21.7879km
INFO:root:Total Time of all routes: 2502.3min
INFO:root:Optmization Complete: Took 00min 06sec to optimize [['East'], ['West']]


In [14]:
import datetime
import matplotlib
from matplotlib import pyplot
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
pyplot.rcParams["figure.figsize"] = [3,7]
for route in route_dict:
    distances = [0]+route_dict[route]['current_distance'][:-1]
    times = route_dict[route]['time']
    distances = np.array(distances).cumsum()
    times = np.array(times)

    y_ticks = [f'{name} - {datetime.timedelta(seconds=float(one_time))}' for one_time,name in zip(times,route_dict[route]['current_names'])]
    pyplot.yticks(ticks=times, labels=y_ticks)
    pyplot.plot(distances, times, marker='o')
    pyplot.show()

In [ ]:
pyplot.rcParams["figure.figsize"] = [3,7]
for route in route_dict:
    distances = pd.Series(route_dict[route]['travel_time']).shift(1).values[1:]
    times = pd.Series(route_dict[route]['time']).diff()[1:]-150
    times = (times-distances)
    times = [int(t) for t in times]
    #distances = np.array(distances).cumsum()
    times = np.array(times)

    y_ticks = [f'{name[0]} to {name[1]} - {datetime.timedelta(seconds=float(one_time))}' for one_time,name in zip(times,zip(route_dict[route]['current_names'], route_dict[route]['next_names']))]
    pyplot.yticks(ticks=times, labels=y_ticks)
    pyplot.plot(times, marker='o')
    #pyplot.plot(distances, marker='o')
    pyplot.show()